In [ ]:
import os
import sys
import math

import numpy
import pandas

# Generalized matrix operations:

def __extractNodes(matrix):
    nodes = set()
    for colKey in matrix:
        nodes.add(colKey)
    for rowKey in matrix.T:
        nodes.add(rowKey)
    return nodes

def __makeSquare(matrix, trustScore, keys, default=0.0):
    matrix = matrix.copy()
    def insertMissingColumns(matrix):
        for key in keys:
            if not key in matrix:
                matrix[key] = pandas.Series(default, index=matrix.index) + pandas.Series(default, index=trustScore.index)
        return matrix
    matrix = insertMissingColumns(matrix) # insert missing columns
    matrix = insertMissingColumns(matrix.T).T # insert missing rows
    return matrix.fillna(default)


def __ensureRowsPositive(matrix,trustScore):
    matrix = matrix.T
    for colKey in matrix:
        if matrix[colKey].sum() == 0.0:
            matrix[colKey] = pandas.Series(numpy.ones(len(matrix[colKey])), index=matrix.index) + (1- pandas.Series(numpy.ones(len(trustScore[colKey])))
    return matrix.T


def __normalizeRows(matrix,trustScore):
    return matrix.div(matrix.sum(axis=1) * trustScore.sum(axis=1), axis=0)

def __euclideanNorm(series):
    return math.sqrt(series.dot(series))

# PageRank specific functionality:

def __startState(nodes):
    if len(nodes) == 0: raise ValueError("There must be at least one node.")
    startProb = 1.0 / float(len(nodes))
    return pandas.Series({node : startProb for node in nodes})

def __integrateRandomSurfer(nodes, transitionProbabilities, rsp):
    alpha = 1.0 / float(len(nodes)) * rsp
    return transitionProbabilities.copy().multiply(1.0 - rsp) + alpha

def powerIteration(transitionWeights, trustScore, rsp=0.15, epsilon=0.00001, maxIterations=1000):
    # Clerical work:
    transitionWeights = pandas.DataFrame(transitionWeights)
    trustScore = pandas.DataFrame(trustScore)
    nodes = __extractNodes(transitionWeights, trustScore)
    transitionWeights = __makeSquare(transitionWeights, trustScore, nodes, default=0.0)
    transitionWeights = __ensureRowsPositive(transitionWeights, trustScore)
    # Setup:
    state = __startState(nodes)
    transitionProbabilities = __normalizeRows(transitionWeights, trustScore)
    transitionProbabilities = __integrateRandomSurfer(nodes, transitionProbabilities, rsp) 
    # Power iteration:
    for iteration in range(maxIterations):
        oldState = state.copy()
        state = state.dot(transitionProbabilities)
        delta = state - oldState
        if __euclideanNorm(delta) < epsilon: 
            break
    return state
